<a href="https://colab.research.google.com/github/sankarvinayak/DL-assignment1/blob/main/DL_assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1 DA6401 Deeplearning
## Sankar Vinayak E P

In [1]:
!pip install wandb
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs24m041 (cs24m041-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
import wandb
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
import numpy as np
np.random.seed(42)

## Question 1
Plot fasion-mnist dataset


In [ ]:
wandb.init(project="CS6401")
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

wandb: Currently logged in as: cs24m041 (cs24m041-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
train_labels

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

In [26]:
import wandb
import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import fashion_mnist
run=wandb.init(project="fmnist")
(train_images, train_labels), _ = fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

sample_images = {}
for label in range(10):
    idx = np.where(train_labels == label)[0][0]
    sample_images[label] = train_images[idx]

list_log = []
for label in sorted(sample_images.keys()):
    caption = class_names[label]
    im = sample_images[label]
    list_log.append(wandb.Image(im, caption=caption))
wandb.log({"fasion mnist each class samples": list_log})

wandb.finish()


## Question 2
Impliment feedforward neural network

#### Actvation functions

In [3]:
from abc import abstractmethod
class ActivationFn:
  @abstractmethod
  def forward(self,X):
      pass
  @abstractmethod
  def grad(self,X):
      pass
class Sigmoid(ActivationFn):
  def forward(self, X):
      return 1/(1 + np.exp(-X))
  def grad(self,X):
      forward_x = self.forward(X)
      return forward_x*(1 - forward_x)
class Tanh(ActivationFn):
  def forward(self,X):
    return np.tanh(X)
  def grad(self,X):
    return 1-self.forward(X)**2
class ReLU(ActivationFn):
  def forward(self,X):
    return np.maximum(0,X)
  def grad(self,X):
    return np.where(X > 0, 1, 0)
class Softmax(ActivationFn):
  def forward(self,X):
    X=X-np.max(X,axis=1,keepdims=True)
    # print(np.sum(X))
    exps=np.exp(X)
    return exps/np.sum(exps,axis=1,keepdims=True)
  def grad():
    pass
class Linear(ActivationFn):
  def forward(self,X):
    return X
  def grad():
    return 1

#### Loss functions

In [4]:
def MSE(prediction,actual):
  assert prediction.shape==actual.shape, "Shapes of prediction and actual must be the same"
  diff=prediction-actual
  sqr=diff**2
  return np.mean(sqr)
# def CrossEntropy(prediction,actual):
#   return -np.sum(actual*np.log(prediction)+1e-8,axis=1)
#   # return -np.log(prediction[np.arange(len(prediction)), np.argmax(actual, axis=1)])

def CrossEntropy(prediction, actual):
    # Clip predictions to avoid log(0)
    prediction = np.clip(prediction, 1e-15, 1.0)
    return -np.sum(actual * np.log(prediction), axis=1)

In [5]:
def one_hot_encode(labels, num_classes):
    return np.eye(num_classes)[labels]

In [6]:
def accuracy(y_true,y_pred):
  y_pred=np.argmax(y_pred,axis=1)
  return np.mean(y_pred==y_true)

#### Layer

In [7]:
class fc_layer:
  def __init__(self,n_inputs,n_output,activation_fn:ActivationFn=Sigmoid(),initialization="random"):
    if initialization=="Xavier":
      pass
    else:
      self.weights=np.random.randn(n_inputs, n_output)*0.1
      self.bias=np.random.randn(1, n_output)
    self.grad_w=np.zeros_like(self.weights)
    self.grad_b=np.zeros_like(self.bias)
    self.u_w=np.zeros_like(self.weights)
    self.u_b=np.zeros_like(self.bias)
    self.activation_fn=activation_fn
  def forward_pass(self, h_prev):
    self.h_prev=h_prev
    self.a_layer=np.dot(h_prev,self.weights)+self.bias
    self.h_this=self.activation_fn.forward(self.a_layer)
    return self.h_this
  def zeroGrad(self):
    self.grad_w=np.zeros_like(self.weights)
    self.grad_b=np.zeros_like(self.bias)

#### Network

In [8]:
class Network:
  def __init__(self):
    self.layers=[]
  def append_layer(self,layer:fc_layer):
    self.layers.append(layer)
  def forward_pass_network(self,X):
    self.input=X
    temp=X
    # print(temp.shape)
    for layer in self.layers:
        temp=layer.forward_pass(temp)
    self.output=temp
    return self.output
  def calculate_grad(self,Y_hat,Y_true):
    batch_size=Y_true.shape[0]
    grad_aL=-(Y_true-Y_hat)/batch_size
    grad_ak=grad_aL
    for k in reversed(range(len(self.layers))):
        layer=self.layers[k]
        if k==0:
            h_prev=self.input
        else:
            h_prev=self.layers[k-1].h_this
        layer.grad_w=np.dot(h_prev.T,grad_ak)
        layer.grad_b=np.sum(grad_ak,axis=0,keepdims=True)
        if k > 0:
            grad_h=np.dot(grad_ak, layer.weights.T)
            prev_layer=self.layers[k-1]
            grad_ak=grad_h*prev_layer.activation_fn.grad(prev_layer.a_layer)
  def zero_grad(self):
    for layer in self.layers:
        layer.zeroGrad()

In [8]:
np.random.seed(1)
nn=Network()
fc_layer1=fc_layer(n_inputs=2,n_output=3)
fc_layer2=fc_layer(n_inputs=3,n_output=2,activation_fn=Softmax())
nn.append_layer(fc_layer1)
nn.append_layer(fc_layer2)
X=np.array([[1,2],[3,4],[5,6]])
print(X.shape)
Y=nn.forward_pass_network(X)
print(Y)

(3, 2)
[[0.23144163 0.76855837]
 [0.23426373 0.76573627]
 [0.23647384 0.76352616]]


In [ ]:
CrossEntropy(Y,np.array([[1,0],[0,1],[0,1]]))

array([3.3214944 , 0.02419299, 0.01866977])

## Question 3
Impliment backprop

In [9]:
def train_loop(X,Y,max_iter:int,model:Network,batch_size:int=None,beta=0.0,loss_fn=CrossEntropy,eta=0.01,print_iter=10,grad_algo=None,gd_type=None):
    n_samples=X.shape[0]
    if batch_size is None:
      batch_size=n_samples
    elif gd_type=="SGD":
      batch_size=1
    for epoch in range(max_iter):
      permutation=np.random.permutation(n_samples)
      X_shuffled=X[permutation]
      Y_shuffled=Y[permutation]
      epoch_loss=0
      num_batches=0

      for i in range(0, n_samples, batch_size):
        batch_end=min(i + batch_size, n_samples)
        X_batch=X_shuffled[i:batch_end]
        Y_batch=Y_shuffled[i:batch_end]
        if grad_algo==None:
          batch_loss=vanilla_gradient_descent(X_batch,Y_batch,model,loss_fn,beta,eta)
        elif grad_algo=="momentum":
          batch_loss=momentum_based_gradient_descent(X_batch,Y_batch,model,loss_fn,beta,eta)
        elif grad_algo=="nesterov":
          batch_loss=nestrov_accelerated_gradient_descent(X_batch,Y_batch,model,loss_fn,beta,eta)
        num_batches += 1
        epoch_loss+=batch_loss

      epoch_loss/=num_batches
      if epoch%print_iter==0:
          print(f"Epoch {epoch}: Loss = {epoch_loss:.4f}")

def vanilla_gradient_descent(X_batch,Y_batch,model:Network,loss_fn,beta,eta):
  model.zero_grad()
  Y_hat=model.forward_pass_network(X_batch)
  batch_loss_vec=loss_fn(Y_hat,Y_batch)
  batch_loss=np.mean(batch_loss_vec)
  model.calculate_grad(Y_hat,Y_batch)
  for layer in model.layers:
    layer.u_w=beta*layer.u_w+eta*layer.grad_w
    layer.u_b=beta*layer.u_b+eta*layer.grad_b
    layer.weights-=layer.u_w
    layer.bias-=layer.u_b
  return batch_loss

In [ ]:
# Create a dummy dataset for classification
n_samples = 100    # number of samples
n_features = 4       # input features
n_outputs = 3        # number of classes

X = np.random.randn(n_samples, n_features)

# Generate dummy one-hot encoded labels:
Y_indices = np.random.randint(0, n_outputs, size=n_samples)
Y = np.zeros((n_samples, n_outputs))
Y[np.arange(n_samples), Y_indices] = 1
# Build the network:
net = Network()
net.append_layer(fc_layer(n_inputs=n_features, n_output=5, activation_fn=Sigmoid()))
net.append_layer(fc_layer(n_inputs=5, n_output=n_outputs, activation_fn=Softmax()))

# Train using gradient descent with momentum:
train_loop(X, Y, max_iter=10, model=net,  loss_fn=CrossEntropy, eta=0.01)
print(X.shape,Y.shape)


Epoch 0: Loss = 1.3191
(100, 4) (100, 3)


#### Fasion Mnist

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


(60000, 28, 28)

((60000, 784), (10000, 784), (60000,), (10000,), 10)

<ipython-input-2-20d5c034241c>:11: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-X))


Epoch 0: Loss = 2.6389
Epoch 10: Loss = 2.3533
Epoch 20: Loss = 2.3101
Epoch 30: Loss = 2.3032
Epoch 40: Loss = 2.3021
Epoch 50: Loss = 2.3018
Epoch 60: Loss = 2.3016
Epoch 70: Loss = 2.3014
Epoch 80: Loss = 2.3013
Epoch 90: Loss = 2.3011


<ipython-input-2-20d5c034241c>:11: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-X))


0.21468333333333334

### Gradient Descent

### Stochastic Gradient descent

In [ ]:

# Build the network:
net = Network()
net.append_layer(fc_layer(n_inputs=n_features, n_output=5, activation_fn=Sigmoid()))
net.append_layer(fc_layer(n_inputs=5, n_output=n_outputs, activation_fn=Softmax()))


# SGD
train_loop(X, Y, max_iter=10, model=net,  loss_fn=CrossEntropy, eta=0.01,gd_type="SGD")



Epoch 0: Loss = 1.3723


### Moementum based Gradient descent

In [10]:
def momentum_based_gradient_descent(X_batch,Y_batch,model:Network,loss_fn,beta,eta):
  return vanilla_gradient_descent(X_batch,Y_batch,model,loss_fn,beta,eta)

In [ ]:

net = Network()
net.append_layer(fc_layer(n_inputs=n_features, n_output=5, activation_fn=Sigmoid()))
net.append_layer(fc_layer(n_inputs=5, n_output=n_outputs, activation_fn=Softmax()))


# Train using gradient descent with momentum:
train_loop(X, Y, max_iter=100, model=net,  loss_fn=CrossEntropy, eta=0.01,beta=0.8,grad_algo="momentum")



Epoch 0: Loss = 1.3796
Epoch 10: Loss = 1.2808
Epoch 20: Loss = 1.1870
Epoch 30: Loss = 1.1367
Epoch 40: Loss = 1.1127
Epoch 50: Loss = 1.1020
Epoch 60: Loss = 1.0975
Epoch 70: Loss = 1.0956
Epoch 80: Loss = 1.0948
Epoch 90: Loss = 1.0944


### Nestrov accelerated gradient descent

In [11]:
# def nestrov_accelerated_gradient_descent(X,Y,max_iter:int,model:Network,batch_size:int=None,beta=0.0,loss_fn=CrossEntropy,eta=0.01):
#     n_samples=X.shape[0]
#     if batch_size is None:
#       batch_size=n_samples
#     for epoch in range(max_iter):
#       permutation=np.random.permutation(n_samples)
#       X_shuffled=X[permutation]
#       Y_shuffled=Y[permutation]
#       epoch_loss=0
#       num_batches=0
#       for i in range(0, n_samples, batch_size):
#         batch_end=min(i + batch_size, n_samples)
#         X_batch=X_shuffled[i:batch_end]
#         Y_batch=Y_shuffled[i:batch_end]
#         batch_loss=nestrov_accelerated_gradient_descent(X_batch,Y_batch,model:Network,loss_fn,beta,eta)
#       epoch_loss/=num_batches
#       if epoch%100==0:
#           print(f"Epoch {epoch}: Loss = {epoch_loss:.4f}")
def nestrov_accelerated_gradient_descent(X_batch,Y_batch,model:Network,loss_fn,beta,eta):
    model.zero_grad()
    for layer in model.layers:
      layer.weights-=beta*layer.u_w
      layer.bias-=beta*layer.u_b
    Y_hat=model.forward_pass_network(X_batch)
    batch_loss_vec=loss_fn(Y_hat,Y_batch)
    batch_loss=np.mean(batch_loss_vec)
    model.calculate_grad(Y_hat,Y_batch)
    for layer in model.layers:
      layer.u_w=beta*layer.u_w+eta*layer.grad_w
      layer.u_b=beta*layer.u_b+eta*layer.grad_b
      layer.weights-=layer.u_w
      layer.bias-=layer.u_b
    # print(batch_loss)
    return batch_loss

In [ ]:

net = Network()
net.append_layer(fc_layer(n_inputs=n_features, n_output=5, activation_fn=Sigmoid()))
net.append_layer(fc_layer(n_inputs=5, n_output=n_outputs, activation_fn=Softmax()))


# Train using gradient descent with momentum:
train_loop(X, Y, max_iter=10, model=net,  loss_fn=CrossEntropy, eta=0.01,beta=0.8,grad_algo="nesterov")



Epoch 0: Loss = 1.6038


### RMSProp

### adam

### nadam

## Question 4
Best hyper parameter value

- number of epochs: 5, 10
- number of hidden layers:  3, 4, 5
- size of every hidden layer:  32, 64, 128
- weight decay (L2 regularisation): 0, 0.0005,  0.5
- learning rate: 1e-3, 1 e-4
- optimizer:  sgd, momentum, nesterov, rmsprop, adam, nadam
- batch size: 16, 32, 64
- weight initialisation: random, Xavier
- activation functions: sigmoid, tanh, ReLU

In [12]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train.shape

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


(60000, 28, 28)

In [13]:
x_train_flat=x_train.reshape(x_train.shape[0],-1)
x_test_flat=x_test.reshape(x_test.shape[0],-1)
x_train_flat.shape,x_test_flat.shape,y_train.shape,y_test.shape,np.unique(y_train).shape[0]

((60000, 784), (10000, 784), (60000,), (10000,), 10)

In [14]:
inp_shp=x_train_flat.shape[1]
out_shp=np.unique(y_train).shape[0]
one_hot_y_train=one_hot_encode(y_train,out_shp)

In [15]:

net = Network()
net.append_layer(fc_layer(n_inputs=inp_shp, n_output=128, activation_fn=Tanh()))
net.append_layer(fc_layer(n_inputs=128, n_output=128, activation_fn=Tanh()))
net.append_layer(fc_layer(n_inputs=128, n_output=128, activation_fn=Tanh()))
net.append_layer(fc_layer(n_inputs=128, n_output=out_shp, activation_fn=Softmax()))

In [42]:
train_loop(X=x_train_flat, Y=one_hot_y_train, max_iter=100, batch_size=512,model=net,beta=0.9,  loss_fn=CrossEntropy, eta=0.005,print_iter=10,grad_algo="momentum")

Epoch 0: Loss = 1.2684
Epoch 10: Loss = 0.6675
Epoch 20: Loss = 0.6604
Epoch 30: Loss = 0.6072
Epoch 40: Loss = 0.6705
Epoch 50: Loss = 0.5774
Epoch 60: Loss = 0.6251
Epoch 70: Loss = 0.5986
Epoch 80: Loss = 0.6113
Epoch 90: Loss = 0.5783


In [16]:
y_pred=net.forward_pass_network(x_train_flat)
accuracy(y_train,y_pred)

<ipython-input-2-abd793ab2455>:11: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-X))


0.8361333333333333

In [17]:
y_pred=net.forward_pass_network(x_test_flat)
accuracy(y_test,y_pred)

<ipython-input-2-abd793ab2455>:11: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-X))


0.8154

In [15]:
import wandb
import datetime
import time

wandb: Currently logged in as: cs24m041 (cs24m041-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [22]:
log_accuracy_entry(0.8361333333333333)

In [16]:
import wandb

sweep_config = {
    "method": "grid",
    "metric": {
        "name": "val_accuracy",
        "goal": "maximize"
    },
    "parameters": {
        "learning_rate": {
            "values": [0.1, 0.01]
        },
        "epochs": {
            "values": [10, 50]
        },
        "batch_size": {
            "values": [256, 512]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="sweep_trail_run")
print("Sweep ID:", sweep_id)


Create sweep with ID: 55v73ha0
Sweep URL: https://wandb.ai/cs24m041-iit-madras/sweep_trail_run/sweeps/55v73ha0
Sweep ID: 55v73ha0


In [17]:
def wandtrain(config=None):
    with wandb.init(config=config):
      config = wandb.config
      lr=config.learning_rate
      epochs=config.epochs
      batch_size=config.batch_size
      train_loop(X=x_train_flat, Y=one_hot_y_train, max_iter=epochs, batch_size=batch_size,model=net,beta=0.9,  loss_fn=CrossEntropy, eta=0.005,print_iter=10,grad_algo="momentum")
      y_pred=net.forward_pass_network(x_test_flat)
      train_accuracy=accuracy(y_test,y_pred)
      wandb.log({"train_accuracy":train_accuracy})

In [ ]:
wandb.agent(sweep_id, wandtrain, count=5)

wandb: Agent Starting Run: yuiwn137 with config:
wandb: 	batch_size: 256
wandb: 	epochs: 50
wandb: 	learning_rate: 0.1


Epoch 0: Loss = 0.8239
Epoch 10: Loss = 0.7553
Epoch 20: Loss = 0.7132
Epoch 30: Loss = 0.7279
Epoch 40: Loss = 0.6695


train_accuracy,▁
train_accuracy,0.7362


wandb: Agent Starting Run: v8i1jomx with config:
wandb: 	batch_size: 256
wandb: 	epochs: 50
wandb: 	learning_rate: 0.01


Epoch 0: Loss = 0.6519
Epoch 10: Loss = 0.6488
Epoch 20: Loss = 0.6065
Epoch 30: Loss = 0.6016
Epoch 40: Loss = 0.6108


...

wandb: Agent Starting Run: 8429a6jw with config:
wandb: 	batch_size: 512
wandb: 	epochs: 10
wandb: 	learning_rate: 0.01


Epoch 0: Loss = 0.5633


train_accuracy,▁
train_accuracy,0.776


wandb: Agent Starting Run: hbaayt91 with config:
wandb: 	batch_size: 512
wandb: 	epochs: 50
wandb: 	learning_rate: 0.1


Epoch 0: Loss = 0.5533


In [ ]:
wandb.finish()

## Question 5

Best accuracy on validation set across all models

## Question 6

inferences

## Question 7

Model with best accuracy on test set ,
Plot confusion matrix

## Question 8

Compare MSE with cross entropy

## Quetion 9
Link to github:https://github.com/sankarvinayak/DL-assignment1

## Question 10
Recomendation of 3 set of hyperparameter for MNIST